<a href="https://colab.research.google.com/github/Ducktard/Text2KnowledgeGraph/blob/dev/text3KnowledgeGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 2.4 MB/s eta 0:00:00


In [2]:
!wget https://raw.githubusercontent.com/bundestag/gesetze/refs/heads/master/h/hgb/index.md

--2025-06-06 11:18:45--  https://raw.githubusercontent.com/bundestag/gesetze/refs/heads/master/h/hgb/index.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 776684 (758K) [text/plain]
Saving to: ‘index.md’

index.md            100%[===================>] 758.48K  --.-KB/s    in 0.06s   

2025-06-06 11:18:45 (12.3 MB/s) - ‘index.md’ saved [776684/776684]



In [3]:
with open("index.md", encoding='utf-8') as file:
    hgb_text = file.read()

In [4]:
len(hgb_text)

762796

In [5]:
!wget https://raw.githubusercontent.com/Ducktard/Text2KnowledgeGraph/refs/heads/main/koalition.txt
with open("koalition.txt", encoding='utf-8') as file:
    koalititionsvertrag_text = file.read()


--2025-06-06 11:20:45--  https://raw.githubusercontent.com/Ducktard/Text2KnowledgeGraph/refs/heads/main/koalition.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548072 (535K) [text/plain]
Saving to: ‘koalition.txt’

koalition.txt       100%[===================>] 535.23K  --.-KB/s    in 0.05s   

2025-06-06 11:20:45 (10.7 MB/s) - ‘koalition.txt’ saved [548072/548072]



In [6]:
koalititionsvertrag_text

'Deutschlands Zukunft gestalten\nKoalitionsvertrag\nzwischen CDU, CSU und SPD\n18. Legislaturperiode\nGliederung 2\n1. Wachstum, Innovation und Wohlstand.......................................................13\n1.1. Deutschlands Wirtschaft stärken ................................................................13\nUnsere Strategie für nachhaltigen Fortschritt..................................................13\nEuropäische Wirtschaftspolitik ........................................................................15\nAußenwirtschaft ..............................................................................................15\nWettbewerbsrecht ...........................................................................................17\nTransparenz bei Managergehältern ................................................................17\nRohstoffsicherung ...........................................................................................17\nIndustrie ...........................

In [ ]:
{\n"
          '       node_1: Required, an entity object with attributes: {"label": "as per the ontology", "name": "Name of the entity"},\n'
          '       node_2: Required, an entity object with attributes: {"label": "as per the ontology", "name": "Name of the entity"},\n'
          "       relationship: Describe the relationship between node_1 and node_2 as per the context, in one word.\n"
          "   }


from pydantic import BaseModel, Field
from typing import List
class NodeResponse(BaseModel):
    label: str = Field(description="This is the Name of the city")
    name: str = Field(description="This is the country of the city")

class EdgeResponse(BaseModel):
    node_1: str = Field(description="This is the Name of the city")
    node_2: str = Field(description="This is the country of the city")
    relationship: str = Field(description="This is the country of the city")

class Edges(BaseModel):
    city: List[EdgeResponse]

In [7]:
import groq
from google.colab import userdata

groq_api_key = userdata.get('groq_api_key')

client = groq.Groq(
    api_key=groq_api_key,
)

models = {
    "llama3-70b-8192": {
        "name": "LLaMA3-70b-Instruct",
        "tokens": 8192,
        "developer": "Meta",
    },
    "llama3-8b-8192": {
        "name": "LLaMA3-8b-Instruct",
        "tokens": 8192,
        "developer": "Meta",
    },
    "mixtral-8x7b-32768": {
        "name": "Mixtral-8x7b-Instruct-v0.1",
        "tokens": 32768,
        "developer": "Mistral",
    },
    "gemma-7b-it": {"name": "Gemma-7b-it", "tokens": 8192, "developer": "Google"},
}

In [ ]:
_ontology = """
        {"Person": "Person name without any adjectives"},
        "Place",
        "Object",
        "Document",
        "Concept",
        "Organisation",
        "Event",
        "Action",
    ],
    relationships=["Relationship between Any two labeled entities"],
"""

In [ ]:
def splitter(n, s):
    pieces = s.split()
    return (" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n))

In [ ]:
def system_message(sourcetext) -> str:
      return (
          "You are an expert at creating Knowledge Graphs. "
          "Consider the following ontology. \n"
          f"{_ontology} \n"
          "The user will provide you with an input text delimited by ```. "
          "Extract all the entities and relationships from the user-provided text as per the given ontology. Do not use any previous knowledge about the context."
          "Remember there can be multiple direct (explicit) or implied relationships between the same pair of nodes. "
          "Be consistent with the given ontology. Use ONLY the labels and relationships mentioned in the ontology. "
          "Format your output as a json with the following schema. \n"
          "[\n"
          "   {\n"
          '       node_1: Required, an entity object with attributes: {"label": "as per the ontology", "name": "Name of the entity"},\n'
          '       node_2: Required, an entity object with attributes: {"label": "as per the ontology", "name": "Name of the entity"},\n'
          "       relationship: Describe the relationship between node_1 and node_2 as per the context, in one word.\n"
          "   },\n"
          "]\n"
          "Do not add any other comment before or after the json. Respond ONLY with a well formed json that can be directly read by a program."
          "``` \n"
          f"{sourcetext} \n"
          " ```"
      )

def quality_message(sourcetext) -> str:
      return (

          "You are an expert at creating Knowledge Graphs. please check the following JSON for syntax errors and correct them:"
          f"{sourcetext} \n"

      )

def prompt_ontology_generation(sourcetext) -> str:
      return (

          "You are an expert at creating Knowledge Graphs. "
          "The user will provide you with an input text delimited by ```. "

          "please extract the ontoly  of the input text\n"
          "``` \n"
          f"{sourcetext} \n"
          " ```"
          "Respond ONLY with a well formed python list that can be directly read by a program."
      )

# TODO:  Wie gross kann splitter argument sein bevor max prompt input erreicht

In [ ]:
import json
import tqdm
json_result = []
counter=0
for chunk in tqdm.tqdm(splitter(500, koalititionsvertrag_text)):


  tempmessage = system_message(chunk)
  chat_completion = client.chat.completions.create(
            model='llama3-70b-8192',
            messages=[{"role": "user", "content": tempmessage}],
            max_tokens=2048,temperature=0.0
        )
  qualitymessage = quality_message(chat_completion.choices[0].message.content)
  chat_completion = client.chat.completions.create(
            model='llama3-70b-8192',
            messages=[{"role": "user", "content": qualitymessage}],
            max_tokens=2048,temperature=0.0
        )
  if len(chat_completion.choices[0].message.content) > 0:
    json_result.extend(json.loads(chat_completion.choices[0].message.content))
  counter+=1

  print(counter)
  if len(json_result)>10:
    break

0it [00:12, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json
import tqdm
ontology_list = []
counter=0
for chunk in tqdm.tqdm(splitter(1500, koalititionsvertrag_text)):
  ontology_prompt = prompt_ontology_generation(chunk)
  chat_completion = client.chat.completions.create(
            model='llama3-70b-8192',
            messages=[{"role": "user", "content": ontology_prompt}],
            max_tokens=2048,temperature=0.0
        )

  if len(chat_completion.choices[0].message.content) > 0:
    ontology_list.append(chat_completion.choices[0].message.content)
  counter+=1

  print(counter)

1it [00:06,  6.08s/it]

1


2it [00:31, 17.53s/it]

2


3it [01:11, 27.79s/it]

...

41


42it [30:19, 44.20s/it]

42


43it [30:37, 42.74s/it]

43


Mounted at /content/gdrive


In [ ]:
import pandas as pd
ontology_df = pd.DataFrame(ontology_list)
ontology_df.to_pickle("/content/gdrive/MyDrive/ontology.pkl")

In [9]:
import pandas as pd
ontology_df = pd.read_pickle("/content/gdrive/MyDrive/ontology.pkl")

In [22]:
ontology_df

,0
0,Here is the extracted ontology in a Python lis...
1,Here is the extracted ontology in a Python lis...
2,Here is the extracted ontology in a Python lis...
3,Here is the ontology extracted from the input ...
4,Here is the extracted ontology in a Python lis...
5,Here is the ontology extracted from the input ...
6,Here is the ontology extracted from the input ...
7,Here is the extracted ontology in a Python lis...
8,Here is the extracted ontology in a Python lis...
9,Here is the ontology extracted from the input ...


In [29]:
ontology_df[0].iloc[0]

'Here is the extracted ontology in a Python list format:\n\n```\n[\n    "Deutschlands Zukunft gestalten",\n    "Wachstum, Innovation und Wohlstand",\n    "Deutschlands Wirtschaft stärken",\n    "Europäische Wirtschaftspolitik",\n    "Außenwirtschaft",\n    "Wettbewerbsrecht",\n    "Transparenz bei Managergehältern",\n    "Rohstoffsicherung",\n    "Industrie",\n    "Mittelstand, Handwerk, Handel und Freie Berufe",\n    "Existenzgründer und Wachstumsfinanzierung",\n    "Regionale Strukturpolitik – Deutsche Einheit stärken",\n    "Kultur- und Kreativwirtschaft",\n    "Tourismus",\n    "Unternehmensnachfolge",\n    "Rechtsrahmen",\n    "Bürokratieabbau und bessere Rechtsetzung",\n    "In Deutschlands Zukunft investieren: Bildung und Forschung",\n    "Hochschulen",\n    "Allgemeine Bildung",\n    "Berufliche Bildung",\n    "Forschung",\n    "Fachkräftesicherung",\n    "Teilhabe von Zuwanderern stärken",\n    "In Deutschlands Zukunft investieren: Infrastruktur",\n    "Verkehr",\n    "Digital

In [40]:
list_str = '"lista","tool"'
_list = list(list_str)
_list

TypeError: 'list' object is not callable

In [52]:
ontology_df = ontology_df[0].str.split('[').str[1].str.replace('\n','').str.replace('  ','').str.replace('  ','').str.replace('"','').str.replace(']```','').str.replace(', ','').str.split(',')

In [55]:
ontology_df

,0
0,"[Deutschlands Zukunft gestalten, WachstumInnov..."
1,[]
2,"[Herausforderungen, Innovation, Investitionen,..."
3,"[Rohstoffstrategie, Rohstoffeffizienz, Substit..."
4,[]
5,"[Education, Science, Innovation, Economy, Inte..."
6,"[MINTFächern, Informatikunterricht, Grundschul..."
7,[]
8,"[{Entity: StipendienprogrammType: Program}, {E..."
9,"[{Entity: EEGUmlageType: Organization}, {Entit..."


In [ ]:
filename = "output.txt"
with open(filename, 'w', encoding='unicode-escape') as f:
    f.write(chat_completion.choices[0].message.content)

In [ ]:
json.loads(chat_completion.choices[0].message.content)

JSONDecodeError: Unterminated string starting at: line 188 column 51 (char 7179)

In [ ]:
chat_completion.choices[0].message.content

'I\'ve reviewed the JSON data, and I found a syntax error in the last object:\n\n```\n{\n    "node_1": {"label": "Concept", "name": " \n}\n```\n\nThe `name` property is not closed, and the object is not properly closed either.\n\nHere is the corrected JSON data:\n\n```\n[\n    {\n        "node_1": {"label": "Document", "name": "Koalitionsvertrag"},\n        "node_2": {"label": "Organisation", "name": "CDU"},\n        "relationship": "between"\n    },\n    {\n        "node_1": {"label": "Document", "name": "Koalitionsvertrag"},\n        "node_2": {"label": "Organisation", "name": "CSU"},\n        "relationship": "between"\n    },\n    {\n        "node_1": {"label": "Document", "name": "Koalitionsvertrag"},\n        "node_2": {"label": "Organisation", "name": "SPD"},\n        "relationship": "between"\n    },\n    {\n        "node_1": {"label": "Event", "name": "18. Legislaturperiode"},\n        "node_2": {"label": "Document", "name": "Koalitionsvertrag"},\n        "relationship": "of"\n

In [ ]:
prompt_test = """
Bitte Beantworte auf deutsch was du zu Wettbewerbsrecht weisst nur auf grundlage der folgenden Informationen:

[[
    {
           "node_1": {"label": "Document", "name": "Koalitionsvertrag"},
           "node_2": {"label": "Organisation", "name": "CDU"},
           "relationship": "between"
       },
       {
           "node_1": {"label": "Document", "name": "Koalitionsvertrag"},
           "node_2": {"label": "Organisation", "name": "CSU"},
           "relationship": "between"
       },
       {
           "node_1": {"label": "Document", "name": "Koalitionsvertrag"},
           "node_2": {"label": "Organisation", "name": "SPD"},
           "relationship": "between"
       },
       {
           "node_1": {"label": "Event", "name": "18. Legislaturperiode"},
           "node_2": {"label": "Document", "name": "Koalitionsvertrag"},
           "relationship": "of"
       },
       {
           "node_1": {"label": "Concept", "name": "Wachstum"},
           "node_2": {"label": "Concept", "name": "Innovation"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Innovation"},
           "node_2": {"label": "Concept", "name": "Wohlstand"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Wohlstand"},
           "node_2": {"label": "Concept", "name": "Wachstum"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Action", "name": "stärken"},
           "node_2": {"label": "Concept", "name": "Deutschlands Wirtschaft"},
           "relationship": "of"
       },
       {
           "node_1": {"label": "Concept", "name": "Europäische Wirtschaftspolitik"},
           "node_2": {"label": "Concept", "name": "Außenwirtschaft"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Außenwirtschaft"},
           "node_2": {"label": "Concept", "name": "Wettbewerbsrecht"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Wettbewerbsrecht"},
           "node_2": {"label": "Concept", "name": "Transparenz bei Managergehältern"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Transparenz bei Managergehältern"},
           "node_2": {"label": "Concept", "name": "Rohstoffsicherung"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Rohstoffsicherung"},
           "node_2": {"label": "Concept", "name": "Industrie"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Industrie"},
           "node_2": {"label": "Concept", "name": "Mittelstand"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Mittelstand"},
           "node_2": {"label": "Concept", "name": "Handwerk"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Handwerk"},
           "node_2": {"label": "Concept", "name": "Handel"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Handel"},
           "node_2": {"label": "Concept", "name": "Freie Berufe"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Existenzgründer"},
           "node_2": {"label": "Concept", "name": "Wachstumsfinanzierung"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Wachstumsfinanzierung"},
           "node_2": {"label": "Concept", "name": "Regionale Strukturpolitik"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Regionale Strukturpolitik"},
           "node_2": {"label": "Concept", "name": "Deutsche Einheit"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Deutsche Einheit"},
           "node_2": {"label": "Concept", "name": "Kultur- und Kreativwirtschaft"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Kultur- und Kreativwirtschaft"},
           "node_2": {"label": "Concept", "name": "Tourismus"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Tourismus"},
           "node_2": {"label": "Concept", "name": "Unternehmensnachfolge"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Unternehmensnachfolge"},
           "node_2": {"label": "Concept", "name": "Rechtsrahmen"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Rechtsrahmen"},
           "node_2": {"label": "Concept", "name": "Bürokratieabbau"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Bürokratieabbau"},
           "node_2": {"label": "Concept", "name": "bessere Rechtsetzung"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "bessere Rechtsetzung"},
           "node_2": {"label": "Concept", "name": "Bildung"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Bildung"},
           "node_2": {"label": "Concept", "name": "Forschung"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Forschung"},
           "node_2": {"label": "Concept", "name": "Fachkräftesicherung"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Fachkräftesicherung"},
           "node_2": {"label": "Concept", "name": "Teilhabe von Zuwanderern"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Teilhabe von Zuwanderern"},
           "node_2": {"label": "Concept", "name": "Infrastruktur"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Infrastruktur"},
           "node_2": {"label": "Concept", "name": "Verkehr"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Verkehr"},
           "node_2": {"label": "Concept", "name": "Digitale Infrastruktur"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Digitale Infrastruktur"},
           "node_2": {"label": "Concept", "name": "Energiewende"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Energiewende"},
           "node_2": {"label": "Concept", "name": "Klimaschutz"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Klimaschutz"},
           "node_2": {"label": "Concept", "name": "Effizienz"},
           "relationship": "and"
       },
       {
           "node_1": {"label": "Concept", "name": "Effizienz"},
           "node_2": {"label": "Concept", "name": "Ausbau der erneuerbaren Energien"},
           "relationship": "and"
       }
"""
#prompt_test = prompt_test.replace('Land- und Forstwirtschaft','Voigtlaender')

chat_completion = client.chat.completions.create(
            model='llama3-70b-8192',
            messages=[{"role": "user", "content": prompt_test}],
            max_tokens=2048,temperature=0.0
        )
chat_completion.choices[0].message.content


#

'Basierend auf den bereitgestellten Informationen kann ich folgendes zum Wettbewerbsrecht sagen:\n\nDas Wettbewerbsrecht ist ein Konzept, das in Verbindung mit anderen Konzepten wie Außenwirtschaft, Transparenz bei Managergehältern und Rohstoffsicherung steht. Es scheint, dass das Wettbewerbsrecht im Kontext der europäischen Wirtschaftspolitik und der Außenwirtschaft diskutiert wird.\n\nEs gibt keine weiteren Informationen über das Wettbewerbsrecht in den bereitgestellten Daten.'

In [ ]:
https://github.com/Ducktard/Text2KnowledgeGraph/blob/main/koalitionsvertrag-data.pdf

'\nBitte Beantworte auf deutsch was man bei Voigtlaender beachten muss nur auf grundlage der folgenden Informationen:\n\n[\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Event", "name": "1897-05-10"},\n       "relationship": "Ausfertigungsdatum"\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Place", "name": "RGBl"},\n       "relationship": "Fundstelle"\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Event", "name": "2020-12-22"},\n       "relationship": "Zuletzt geändert durch"\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Event", "name": "2021-05-12"},\n       "relationship": "Änderung durch"\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Event", "name": "2021-06-03"},\n  

In [ ]:
prompt_test

In [ ]:
chat_completion.choices[0].message.content

'[\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Event", "name": " Ausfertigungsdatum"},\n       "relationship": "Handelsgesetzbuch was enacted on Ausfertigungsdatum."\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Place", "name": "RGBl"},\n       "relationship": "Handelsgesetzbuch was published in RGBl."\n   },\n   {\n       "node_1": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "node_2": {"label": "Person", "name": "not specified"},\n       "relationship": "Handelsgesetzbuch was modified by an unknown person."\n   },\n   {\n       "node_1": {"label": "Document", "name": "Art. 14 G v. 22.12.2020 I 3256"},\n       "node_2": {"label": "Document", "name": "Handelsgesetzbuch"},\n       "relationship": "Art. 14 G v. 22.12.2020 I 3256 modified Handelsgesetzbuch."\n   },\n   {\n       "node_1": {"label": "Document", "name": "Art. 7 Abs. 2 G v. 12.5.2021 I

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

nx.node_link_graph(json_result)

/usr/local/lib/python3.11/dist-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


AttributeError: 'list' object has no attribute 'get'

In [ ]:
test_prompt_ = """



In [ ]:
chat_completion = client.chat.completions.create(
          model='llama3-70b-8192',
          messages=[{"role": "user", "content": tempmessage}],
          max_tokens=2048
      )

In [ ]:
max_tokens=2048

In [ ]:
prompt_test= """Erstelle eine Tabelle als CSV mit der Spalte Entitaeten,  fuer einen Knowledge Graphen aus dem folgenden TExt:

Make a knowledge graph out of the following paragraphs: #### § 3

(1) Auf den Betrieb der Land- und Forstwirtschaft finden die
Vorschriften des § 1 keine Anwendung.

(2) Für ein land- oder forstwirtschaftliches Unternehmen, das nach Art
und Umfang einen in kaufmännischer Weise eingerichteten
Geschäftsbetrieb erfordert, gilt § 2 mit der Maßgabe, daß nach
Eintragung in das Handelsregister eine Löschung der Firma nur nach den
allgemeinen Vorschriften stattfindet, welche für die Löschung
kaufmännischer Firmen gelten.

(3) Ist mit dem Betrieb der Land- oder Forstwirtschaft ein Unternehmen
verbunden, das nur ein Nebengewerbe des land- oder
forstwirtschaftlichen Unternehmens darstellt, so finden auf das im
Nebengewerbe betriebene Unternehmen die Vorschriften der Absätze 1 und
2 entsprechende Anwendung."""

In [ ]:
prompt_test= """Mache eine Named Entity Recognition und gebe das ergebnis als Liste zurueck auf grundlage des folgenden TExt:

#### § 1

(1) Kaufmann im Sinne dieses Gesetzbuchs ist, wer ein Handelsgewerbe
betreibt.

(2) Handelsgewerbe ist jeder Gewerbebetrieb, es sei denn, daß das
Unternehmen nach Art oder Umfang einen in kaufmännischer Weise
eingerichteten Geschäftsbetrieb nicht erfordert.


#### § 2

Ein gewerbliches Unternehmen, dessen Gewerbebetrieb nicht schon nach §
1 Abs. 2 Handelsgewerbe ist, gilt als Handelsgewerbe im Sinne dieses
Gesetzbuchs, wenn die Firma des Unternehmens in das Handelsregister
eingetragen ist. Der Unternehmer ist berechtigt, aber nicht
verpflichtet, die Eintragung nach den für die Eintragung
kaufmännischer Firmen geltenden Vorschriften herbeizuführen. Ist die
Eintragung erfolgt, so findet eine Löschung der Firma auch auf Antrag
des Unternehmers statt, sofern nicht die Voraussetzung des § 1 Abs. 2
eingetreten ist.

#### § 3

(1) Auf den Betrieb der Land- und Forstwirtschaft finden die
Vorschriften des § 1 keine Anwendung.

(2) Für ein land- oder forstwirtschaftliches Unternehmen, das nach Art
und Umfang einen in kaufmännischer Weise eingerichteten
Geschäftsbetrieb erfordert, gilt § 2 mit der Maßgabe, daß nach
Eintragung in das Handelsregister eine Löschung der Firma nur nach den
allgemeinen Vorschriften stattfindet, welche für die Löschung
kaufmännischer Firmen gelten.

(3) Ist mit dem Betrieb der Land- oder Forstwirtschaft ein Unternehmen
verbunden, das nur ein Nebengewerbe des land- oder
forstwirtschaftlichen Unternehmens darstellt, so finden auf das im
Nebengewerbe betriebene Unternehmen die Vorschriften der Absätze 1 und
2 entsprechende Anwendung."""

In [ ]:
len(prompt_test)

1693

In [ ]:
messages=[{"role": "user", "content": prompt_test}]

In [ ]:
chat_completion = client.chat.completions.create(
            model='llama3-70b-8192',
            messages=messages,
            max_tokens=max_tokens
        )

In [ ]:
print(chat_completion.choices[0].message.content)

Here is the result of the Named Entity Recognition (NER) task on the given text:

**Entities:**

1. **Organizations:**
	* Handelsregister (2 mentions)
2. **Locations:**
	* None
3. **People:**
	* Kaufmann (1 mention)
	* Unternehmer (2 mentions)
4. **Legislation:**
	* Gesetzbuchs (2 mentions)
	* § 1 (2 mentions)
	* § 2 (2 mentions)
	* § 3 (1 mention)

Please note that the entities recognized are based on the context of the text and may not be exhaustive. Additionally, the mentions of each entity are counted separately.


In [ ]:
prompt_2 = """Du sollst einen Knowledge Graphen aus einem Text machen. Gehe dazu in folgenden SChritten vor: Schritt 1: Mache eine Named Entity Recognition und nutze das Ergebnis um im Schritt2  eine CSV TAbelle mit den folgenden Spalten: sourcenode, targetnode, relationship auf grundlage der NER und des textes zu erstellen Der zugrunde liegender TExt::
#### § 1

(1) Kaufmann im Sinne dieses Gesetzbuchs ist, wer ein Handelsgewerbe
betreibt.

(2) Handelsgewerbe ist jeder Gewerbebetrieb, es sei denn, daß das
Unternehmen nach Art oder Umfang einen in kaufmännischer Weise
eingerichteten Geschäftsbetrieb nicht erfordert.


#### § 2

Ein gewerbliches Unternehmen, dessen Gewerbebetrieb nicht schon nach §
1 Abs. 2 Handelsgewerbe ist, gilt als Handelsgewerbe im Sinne dieses
Gesetzbuchs, wenn die Firma des Unternehmens in das Handelsregister
eingetragen ist. Der Unternehmer ist berechtigt, aber nicht
verpflichtet, die Eintragung nach den für die Eintragung
kaufmännischer Firmen geltenden Vorschriften herbeizuführen. Ist die
Eintragung erfolgt, so findet eine Löschung der Firma auch auf Antrag
des Unternehmers statt, sofern nicht die Voraussetzung des § 1 Abs. 2
eingetreten ist.

#### § 3

(1) Auf den Betrieb der Land- und Forstwirtschaft finden die
Vorschriften des § 1 keine Anwendung.

(2) Für ein land- oder forstwirtschaftliches Unternehmen, das nach Art
und Umfang einen in kaufmännischer Weise eingerichteten
Geschäftsbetrieb erfordert, gilt § 2 mit der Maßgabe, daß nach
Eintragung in das Handelsregister eine Löschung der Firma nur nach den
allgemeinen Vorschriften stattfindet, welche für die Löschung
kaufmännischer Firmen gelten.

(3) Ist mit dem Betrieb der Land- oder Forstwirtschaft ein Unternehmen
verbunden, das nur ein Nebengewerbe des land- oder
forstwirtschaftlichen Unternehmens darstellt, so finden auf das im
Nebengewerbe betriebene Unternehmen die Vorschriften der Absätze 1 und
2 entsprechende Anwendung.

unter zuhilfenahme der folgenden Named Entity Recognition: """ + chat_completion.choices[0].message.content

In [ ]:
len(prompt_2)

2507

In [ ]:
messages=[{"role": "user", "content": prompt_2}]

In [ ]:
chat_completion = client.chat.completions.create(
            model='llama3-70b-8192',
            messages=messages,
            max_tokens=max_tokens
        )

In [ ]:
print(chat_completion.choices[0].message.content)

Based on the Named Entity Recognition (NER) results, I will create a CSV table with the following columns: `sourcenode`, `targetnode`, and `relationship`. Here is the resulting table:

| sourcenode | targetnode | relationship |
| --- | --- | --- |
| Kaufmann | Handelsgewerbe | is |
| Handelsgewerbe | Gewerbebetrieb | is |
| Gewerbebetrieb | Unternehmen | is_part_of |
| Unternehmen | Handelsregister | is_registered_in |
| Unternehmer | Handelsgewerbe | operates |
| Handelsgewerbe | Gesetzbuchs | is_defined_by |
| § 1 | Handelsgewerbe | defines |
| § 2 | Handelsgewerbe | defines |
| § 3 | Land- und Forstwirtschaft | excludes |
| Land- und Forstwirtschaft | Unternehmen | is_part_of |
| Unternehmen | Handelsregister | is_registered_in |
| Unternehmer | Nebengewerbe | operates |

Here's a brief explanation of each row:

1. A Kaufmann (merchant) is a type of Handelsgewerbe (commercial enterprise).
2. A Handelsgewerbe is a type of Gewerbebetrieb (business operation).
3. A Gewerbebetrieb is pa

In [ ]:
hgb_text

'---\nTitle: Handelsgesetzbuch\njurabk: HGB\nlayout: default\norigslug: hgb\nslug: hgb\n\n---\n\n# Handelsgesetzbuch (HGB)\n\nAusfertigungsdatum\n:   1897-05-10\n\nFundstelle\n:   RGBl: 1897, 219\n\nZuletzt geändert durch\n:   Art. 14 G v. 22.12.2020 I 3256\n\nÄnderung durch\n:   Art. 7 Abs. 2 G v. 12.5.2021 I 990 (Nr. 23) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 10 G v. 3.6.2021 I 1498 (Nr. 30) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 11 G v. 3.6.2021 I 1534 (Nr. 30) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 4 G v. 16.7.2021 I 3079 (Nr. 47) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 5 G v. 7.8.2021 I 3311 (Nr. 51) textlich nachgewiesen, dokumentarisch noch nicht abschließend bearbeitet\n\nÄnderung durch\n:   Art. 1 G v. 5.7.2021 I 3338 (Nr. 52) textlic

Notizen:
## TODO 04.04.25 result  als networkx graph importieren

## 13.05. Diskussion ueber relationshipo und ob bisheriger output sinnvoll, vorschlag alex: realtionsships festlegen (kaufmann, handelsgewerbe etc. und als nodes nur paragraphen zulassen? Koennte das eine herangehensweise sein um sinnvolle graphen zu erstellen aufs dem hgb?

##23.05.25 Grundsaetzlich auf koalititionsvertrag getestet (findet verbindung auch ueber mehrere kanten und formuliert antwort) Naechstes TODO: Wie bekommen wir die rueckgabe JSONs fehlerfrei/komplett (llm aktuell sehr unzuverlaessig

## 16.05.25 Beim testen ob das befragen von dem knowledge graphen den wir aus dem hgb extrahiert haben funktioniert, gemerkt das das llm sich nicht nur auf den knowledge graph sondern auch auf bereits vorher gelernte infos ueber das hgb bezieht. daher wechsel auf infos die es noch nicht kennt (koalitionsvertrag 2025)

## 30.05. Ideen fuer Problem das chunk verarbeitung nicht ganzen kontext beruecksichtigt:
-->erledigt: ontology liste erstellt.
- mehrere iterationen ueber korpus machen mit verschiedenen geschnittenen chunks
- vorher embedding machen vom korpus und nur die inhaltlich naechsten chunks zum aktuellen chunk mitberuecksichtigen (RAG aehnlich)
- ideen fuer ontology distillieren:
 -> value count auf finale ontology liste

